In [41]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.metrics import Precision, Recall, F1Score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import scipy.stats as stats
import scipy.signal as signal
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [42]:
def normalize(df):
    
    for column in ['x', 'y', 'z']:
        df[column] = (df[column] - df[column].mean()) / df[column].std()
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

def segment(df, window_size, step_size):
    segments = []
    for start in range(0, len(df) - window_size + 1, step_size):
        end = start + window_size
        segment = df.iloc[start:end]
        segments.append(segment)
    return segments

In [43]:
def segment_activities(window_size=32, step_size=16):
    # window_size =   # 2.5 seconds
    # step_size = 20  # 75% overlap

    activities_path = 'Final/Activity' #CHANGE THIS BASED OFF FILE PATH
    activity_segments = {}

    # Iterate over each activity's folder
    for activity_name in os.listdir(activities_path):
        # print(activity_name)
        activity_folder = os.path.join(activities_path, activity_name)
        if os.path.isdir(activity_folder):
            # Store segments for each activity
            activity_segments[activity_name] = []
            
            # Iterate over each CSV file within the activity's folder
            for filename in os.listdir(activity_folder):
                if filename.endswith('.csv'):
                    file_path = os.path.join(activity_folder, filename)
                    
                    # Read  CSV file
                    df = pd.read_csv(file_path)
                    
                    # Normalize data
                    df_normalized = normalize(df)
                    
                    # Segment the data using a rolling window
                    segments = segment(df_normalized, window_size, step_size)
                    
                    # Append the segments to the activity's list
                    activity_segments[activity_name].extend(segments)
    return activity_segments


In [44]:
def fit_model_lstm(activity_segments):
    X, y = [], []

    # Convert the segments into a suitable format for training
    for activity_name, segments in activity_segments.items():
        for segment in segments:
            X.append(segment.to_numpy())  # Assuming segment is a pandas DataFrame
            y.append(activity_name)
    X = np.array(X)
    y = np.array(y)

    # Encode the activity labels into integers
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # Define the LSTM model
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
        LSTM(64),
        Dense(len(le.classes_), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)

    # Predict the labels on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

    # Calculate other classification metrics
    metrics_report = classification_report(y_test, y_pred_classes, output_dict=True)
    accuracy_from_report = accuracy_score(y_test, y_pred_classes)
    print(f"Loss: {loss}")
    print(f"Accuracy: {accuracy}")
    print(f"Accuracy (from report): {accuracy_from_report}")
    print(f"Precision: {metrics_report['weighted avg']['precision']}")
    print(f"Recall: {metrics_report['weighted avg']['recall']}")
    print(f"F1-Score: {metrics_report['weighted avg']['f1-score']}")

In [45]:
def fit_model_rf(activity_segments):
    X, y = [], []
    # Convert the segments into a suitable format for training
    for activity_name, segments in activity_segments.items():
        for segment in segments:
            X.append(segment.to_numpy())  # Assuming segment is a pandas DataFrame
            y.append(activity_name)
    X = np.array(X)
    y = np.array(y)
    print("segments in suitable format")
    # Encode the activity labels into integers
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print("labels encoded")
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    #reshape for rf model
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    #model stuff
    # Define the Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    print("training model")
    # Train the model
    model.fit(X_train, y_train)
    #predict on test data
    y_pred = model.predict(X_test)
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test,y_pred,average = 'macro')
    recall = recall_score(y_test, y_pred, average = 'macro')
    f1 = f1_score(y_test, y_pred, average= 'macro')
    print(f"Model accuracy: {accuracy}")
    print(f"Model precision: {precision}")
    print(f"Model recall: {recall}")
    print(f"Model F1 score: {f1}")
    # return model


In [46]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name = name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(tf.nn.sigmoid(y_pred))  # Use sigmoid to convert the output to between 0 and 1, then round to the nearest integer
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [47]:
def fit_model_SVM(activity_segments):
    X, y = [], []

    for activity_name, segments in activity_segments.items():
        for segment in segments:
            feature_vector = segment.to_numpy().flatten()
            X.append(feature_vector)
            y.append(activity_name)
    print("Segments in suitable format")

    X = np.array(X)
    y = np.array(y)

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    y_encoded = np.where(y_encoded > 0, 1, -1)  # Adjust labels to -1 and 1

    print("Labels encoded")

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size = 0.2, random_state = 42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, input_shape=(X_train_scaled.shape[1],), activation = None) 
    ])

    def hinge_loss(y_true, y_pred):
        return tf.reduce_mean(tf.maximum(1 - y_true * y_pred, 0))

    model.compile(optimizer='sgd', loss=hinge_loss, metrics=['accuracy', Precision(), Recall(), F1Score()])

    print("Compiled model\nTraining Model")
    model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2)

    results = model.evaluate(X_test_scaled, y_test)
    print("Test Results for SVM model - Loss: {:.4f}, Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(*results))
    
    # return model

In [48]:
def fit_model_XGB(activity_segments):
    X, y = [], []


    # Convert the segments into a suitable format for training
    for activity_name, segments in activity_segments.items():
        for segment in segments:
            X.append(segment.to_numpy())  # Assuming segment is a pandas DataFrame
            y.append(activity_name)
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(X.shape[0], -1)
    print("segments in suitable format")


    # Encode the activity labels into integers
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print("labels encoded")
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    print('X_train:', X_train.shape)
    print('y_train:', y_train.shape)
    
    
    # Convert data to DMatrix format for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define parameters for XGBoost
    params = {
        'objective': 'multi:softmax',  # Multi-class classification
        'num_class': 34,                # Number of classes
        'eval_metric': 'merror',         # Evaluation metric: multi-class classification error
        'learning_rate': 0.2, 
        'max_depth': 6, 
        'n_estimators': 200, 
        'subsample': 1.0
    }

# **params
    xgb_classifier = xgb.XGBClassifier(**params)
    xgb_classifier.fit(X_train, y_train)
    # Predict on the test data
    y_pred = xgb_classifier.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average = 'weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f'Test Accuracy: {accuracy}')
    print(f'Test Precision: {precision}')
    print(f'Test Recall: {recall}')
    print(f'Test F1 Score: {f1}')
    
    # return xgb_classifier

In [49]:
def fit_model_lgbm(activity_segments):
    X, y = [], []


    # Convert the segments into a suitable format for training
    for activity_name, segments in activity_segments.items():
        for segment in segments:
            X.append(segment.to_numpy())  # Assuming segment is a pandas DataFrame
            y.append(activity_name)
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(X.shape[0], -1)
    print("segments in suitable format")


    # Encode the activity labels into integers
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print("labels encoded")
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    print('X_train:', X_train.shape)
    print('y_train:', y_train.shape)

    # Model Training
    clf = LGBMClassifier()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average = 'weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f'Test Accuracy: {accuracy}')
    print(f'Test Precision: {precision}')
    print(f'Test Recall: {recall}')
    print(f'Test F1 Score: {f1}')
    
    # return clf


# NO FILTER

In [ ]:
window_sizes = [32, 96, 160]  # 1, 1.5, and 2 seconds
step_sizes_16 = [2, 6, 10]  # 1/16 overlap for each window size
step_sizes_8 = [4, 12, 20]  # 1/8 overlap for each window size
step_sizes_4 = [8, 24, 40]  # 1/4 overlap for each window size
step_sizes = [step_sizes_16 ,step_sizes_8, step_sizes_4]

for index_step in range(3):
    for step in range(3):
        print(f"Step:{step_sizes[index_step][step]}  Window:{window_sizes[step]}")       
        activity_segments = segment_activities(window_sizes[step], step_sizes[index_step][step])
        print(f"Finished  Window:{window_sizes[step]} Segmentation")
        print(f"STARTING LSTM MODEL")
        fit_model_lstm(activity_segments)
        print(f"STARTING RANDOM FOREST MODEL")
        fit_model_rf(activity_segments)
        print(f"STARTING XGB MODEL")
        fit_model_XGB(activity_segments)
        print(f"STARTING SVM MODEL")
        fit_model_SVM(activity_segments)
        print(f"STARTING LGBM MODEL")
        fit_model_lgbm(activity_segments)

# MEAN FILTER

In [ ]:
def windowed_mean(input_df):
    input_x = np.array(input_df.loc[:, "x"])
    input_y = np.array(input_df.loc[:, "y"])
    input_z = np.array(input_df.loc[:, "z"])

    return np.mean(input_x), np.mean(input_y), np.mean(input_z)

def normalize(df):
    
    x, y, z = windowed_mean(df)
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

In [ ]:
window_sizes = [32, 96, 160]  # 1, 1.5, and 2 seconds
step_sizes_16 = [2, 6, 10]  # 1/16 overlap for each window size
step_sizes_8 = [4, 12, 20]  # 1/8 overlap for each window size
step_sizes_4 = [8, 24, 40]  # 1/4 overlap for each window size
step_sizes = [step_sizes_16 ,step_sizes_8, step_sizes_4]

for index_step in range(3):
    for step in range(3):
        print(f"Step:{step_sizes[index_step][step]}  Window:{window_sizes[step]}")       
        activity_segments = segment_activities(window_sizes[step], step_sizes[index_step][step])
        print(f"Finished  Window:{window_sizes[step]} Segmentation")
        print(f"STARTING LSTM MODEL")
        fit_model_lstm(activity_segments)
        print(f"STARTING RANDOM FOREST MODEL")
        fit_model_rf(activity_segments)
        print(f"STARTING XGB MODEL")
        fit_model_XGB(activity_segments)
        print(f"STARTING SVM MODEL")
        fit_model_SVM(activity_segments)
        print(f"STARTING LGBM MODEL")
        fit_model_lgbm(activity_segments)

# STD DEV FITLER

In [ ]:
def windowed_std_dev(input_df):
    input_x = np.array(input_df.loc[:, "x"])
    input_y = np.array(input_df.loc[:, "y"])
    input_z = np.array(input_df.loc[:, "z"])

    return np.std(input_x), np.std(input_y), np.std(input_z)

def normalize(df):
    
    x, y, z = windowed_std_dev(df)
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

In [ ]:
window_sizes = [32, 96, 160]  # 1, 1.5, and 2 seconds
step_sizes_16 = [2, 6, 10]  # 1/16 overlap for each window size
step_sizes_8 = [4, 12, 20]  # 1/8 overlap for each window size
step_sizes_4 = [8, 24, 40]  # 1/4 overlap for each window size
step_sizes = [step_sizes_16 ,step_sizes_8, step_sizes_4]

for index_step in range(3):
    for step in range(3):
        print(f"Step:{step_sizes[index_step][step]}  Window:{window_sizes[step]}")       
        activity_segments = segment_activities(window_sizes[step], step_sizes[index_step][step])
        print(f"Finished  Window:{window_sizes[step]} Segmentation")
        print(f"STARTING LSTM MODEL")
        fit_model_lstm(activity_segments)
        print(f"STARTING RANDOM FOREST MODEL")
        fit_model_rf(activity_segments)
        print(f"STARTING XGB MODEL")
        fit_model_XGB(activity_segments)
        print(f"STARTING SVM MODEL")
        fit_model_SVM(activity_segments)
        print(f"STARTING LGBM MODEL")
        fit_model_lgbm(activity_segments)

# SKEWNESS FILTER

In [3]:
def windowed_skew(input_df):
    input_x = np.array(input_df.loc[:, "x"])
    input_y = np.array(input_df.loc[:, "y"])
    input_z = np.array(input_df.loc[:, "z"])

    return stats.skew(input_x), stats.skew(input_y), stats.skew(input_z)

def normalize(df):
    
    x, y, z = windowed_skew(df)
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

In [ ]:
window_sizes = [32, 96, 160]  # 1, 1.5, and 2 seconds
step_sizes_16 = [2, 6, 10]  # 1/16 overlap for each window size
step_sizes_8 = [4, 12, 20]  # 1/8 overlap for each window size
step_sizes_4 = [8, 24, 40]  # 1/4 overlap for each window size
step_sizes = [step_sizes_16 ,step_sizes_8, step_sizes_4]

for index_step in range(3):
    for step in range(3):
        print(f"Step:{step_sizes[index_step][step]}  Window:{window_sizes[step]}")       
        activity_segments = segment_activities(window_sizes[step], step_sizes[index_step][step])
        print(f"Finished  Window:{window_sizes[step]} Segmentation")
        print(f"STARTING LSTM MODEL")
        fit_model_lstm(activity_segments)
        print(f"STARTING RANDOM FOREST MODEL")
        fit_model_rf(activity_segments)
        print(f"STARTING XGB MODEL")
        fit_model_XGB(activity_segments)
        print(f"STARTING SVM MODEL")
        fit_model_SVM(activity_segments)
        print(f"STARTING LGBM MODEL")
        fit_model_lgbm(activity_segments)

# Kurtosis Filter

In [4]:
def windowed_kurtosis(input_df):
    input_x = np.array(input_df.loc[:, "x"])
    input_y = np.array(input_df.loc[:, "y"])
    input_z = np.array(input_df.loc[:, "z"])

    return stats.kurtosis(input_x), stats.kurtosis(input_y), stats.kurtosis(input_z)

def normalize(df):
    
    x, y, z = windowed_kurtosis(df)
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

In [ ]:
window_sizes = [32, 96, 160]  # 1, 1.5, and 2 seconds
step_sizes_16 = [2, 6, 10]  # 1/16 overlap for each window size
step_sizes_8 = [4, 12, 20]  # 1/8 overlap for each window size
step_sizes_4 = [8, 24, 40]  # 1/4 overlap for each window size
step_sizes = [step_sizes_16 ,step_sizes_8, step_sizes_4]

for index_step in range(3):
    for step in range(3):
        print(f"Step:{step_sizes[index_step][step]}  Window:{window_sizes[step]}")       
        activity_segments = segment_activities(window_sizes[step], step_sizes[index_step][step])
        print(f"Finished  Window:{window_sizes[step]} Segmentation")
        print(f"STARTING LSTM MODEL")
        fit_model_lstm(activity_segments)
        print(f"STARTING RANDOM FOREST MODEL")
        fit_model_rf(activity_segments)
        print(f"STARTING XGB MODEL")
        fit_model_XGB(activity_segments)
        print(f"STARTING SVM MODEL")
        fit_model_SVM(activity_segments)
        print(f"STARTING LGBM MODEL")
        fit_model_lgbm(activity_segments)

# Zero Crossing Rate Filter

In [ ]:
def windowed_zcr(input_df):

    def compute_zcr(input_arr):
        my_array = np.array(input_arr)
        return float(((((my_array[:-1] * my_array[1:]) < 0).sum())/len(input_arr)))

    input_x = np.array(input_df.loc[:, "x"])
    input_y = np.array(input_df.loc[:, "y"])
    input_z = np.array(input_df.loc[:, "z"])

    return compute_zcr(input_x), compute_zcr(input_y), compute_zcr(input_z)

def normalize(df):
    
    x, y, z = windowed_zcr(df)
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

In [ ]:
window_sizes = [32, 96, 160]  # 1, 1.5, and 2 seconds
step_sizes_16 = [2, 6, 10]  # 1/16 overlap for each window size
step_sizes_8 = [4, 12, 20]  # 1/8 overlap for each window size
step_sizes_4 = [8, 24, 40]  # 1/4 overlap for each window size
step_sizes = [step_sizes_16 ,step_sizes_8, step_sizes_4]

for index_step in range(3):
    for step in range(3):
        print(f"Step:{step_sizes[index_step][step]}  Window:{window_sizes[step]}")       
        activity_segments = segment_activities(window_sizes[step], step_sizes[index_step][step])
        print(f"Finished  Window:{window_sizes[step]} Segmentation")
        print(f"STARTING LSTM MODEL")
        fit_model_lstm(activity_segments)
        print(f"STARTING RANDOM FOREST MODEL")
        fit_model_rf(activity_segments)
        print(f"STARTING XGB MODEL")
        fit_model_XGB(activity_segments)
        print(f"STARTING SVM MODEL")
        fit_model_SVM(activity_segments)
        print(f"STARTING LGBM MODEL")
        fit_model_lgbm(activity_segments)

# Dominant Frequency Filter

In [5]:
def dom_f(input_df):

    def compute(input_arr):
        frequency_spectrum = np.fft.fft(input_arr)
        dom = frequency_spectrum[np.argmax(frequency_spectrum)]
        return dom
    input_x = np.array(input_df.loc[:, "x"])
    input_y = np.array(input_df.loc[:, "y"])
    input_z = np.array(input_df.loc[:, "z"])

    return compute(input_x), compute(input_y), compute(input_z)

def normalize(df):
    
    x, y, z = dom_f(df)
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

In [ ]:
window_sizes = [32, 96, 160]  # 1, 1.5, and 2 seconds
step_sizes_16 = [2, 6, 10]  # 1/16 overlap for each window size
step_sizes_8 = [4, 12, 20]  # 1/8 overlap for each window size
step_sizes_4 = [8, 24, 40]  # 1/4 overlap for each window size
step_sizes = [step_sizes_16 ,step_sizes_8, step_sizes_4]

for index_step in range(3):
    for step in range(3):
        print(f"Step:{step_sizes[index_step][step]}  Window:{window_sizes[step]}")       
        activity_segments = segment_activities(window_sizes[step], step_sizes[index_step][step])
        print(f"Finished  Window:{window_sizes[step]} Segmentation")
        print(f"STARTING LSTM MODEL")
        fit_model_lstm(activity_segments)
        print(f"STARTING RANDOM FOREST MODEL")
        fit_model_rf(activity_segments)
        print(f"STARTING XGB MODEL")
        fit_model_XGB(activity_segments)
        print(f"STARTING SVM MODEL")
        fit_model_SVM(activity_segments)
        print(f"STARTING LGBM MODEL")
        fit_model_lgbm(activity_segments)